# Full Automatic

But too complex

In [1]:
!pip install requests

In [4]:
import requests

In [5]:
query = "(Xu, Hua[Author]) AND (School of Biomedical Informatics[Affiliation])"

url_search = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term={query}&retmode=json&retmax=5000"

rsp = requests.get(
    url_search.format(query=query)
)

j = rsp.json()

j['esearchresult']['idlist'][:10]

['37669123',
 '37555837',
 '37390046',
 '37301740',
 '37128366',
 '37018650',
 '36806328',
 '36802713',
 '36417585',
 '36093703']

In [6]:
url_summary = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi?db=pubmed&id={ids}&retmode=json&retmax=5000&version=2.0"

rsp = requests.get(
    url_summary.format(ids=','.join(j['esearchresult']['idlist']))
)

j = rsp.json()

j

{'header': {'type': 'esummary', 'version': '0.3'},
 'result': {'uids': ['37669123',
   '37555837',
   '37390046',
   '37301740',
   '37128366',
   '37018650',
   '36806328',
   '36802713',
   '36417585',
   '36093703',
   '36068551',
   '36033590',
   '35917480',
   '35725267',
   '35709900',
   '37664001',
   '35466079',
   '35426943',
   '35377319',
   '35354802',
   '35311707',
   '35181495',
   '35005421',
   '34457154',
   '34457146',
   '34406805',
   '34272955',
   '34224163',
   '34151987',
   '34081724',
   '34051088',
   '33936520',
   '33795676',
   '33786419',
   '33757278',
   '33674830',
   '33647938',
   '33409263',
   '33317519',
   '33059366',
   '32995818',
   '32930711',
   '32918447',
   '32735224',
   '32626903',
   '32626900',
   '32569358',
   '32537570',
   '32477646',
   '32364754',
   '32308921',
   '32298846',
   '32012237',
   '31816040',
   '31815074',
   '31801529',
   '31801516',
   '31794016',
   '31550346',
   '31438083',
   '31294792',
   '31265066',
 

# Human + Parsing

Download the CSV from PubMed of all results first

In [7]:
import pandas as pd

In [9]:
df = pd.read_csv('csv-XuHuaAutho-set.csv', dtype={
    "Publication Year": int
})
df.head()

,PMID,Title,Authors,Citation,First Author,Journal/Book,Publication Year,Create Date,PMCID,NIHMS ID,DOI
0,31794016,Deep learning in clinical natural language pro...,"Wu S, Roberts K, Datta S, Du J, Ji Z, Si Y, So...",J Am Med Inform Assoc. 2020 Mar 1;27(3):457-47...,Wu S,J Am Med Inform Assoc,2020,2019/12/04,PMC7025365,NaN,10.1093/jamia/ocz200
1,36033590,"The All of Us Research Program: Data quality, ...","Ramirez AH, Sulieman L, Schlueter DJ, Halvorso...",Patterns (N Y). 2022 Aug 12;3(8):100570. doi: ...,Ramirez AH,Patterns (N Y),2022,2022/08/29,PMC9403360,NaN,10.1016/j.patter.2022.100570
2,30943961,Special issue of BMC medical informatics and d...,"Vydiswaran VGV, Zhang Y, Wang Y, Xu H.",BMC Med Inform Decis Mak. 2019 Apr 4;19(Suppl ...,Vydiswaran VGV,BMC Med Inform Decis Mak,2019,2019/04/05,PMC6448180,NaN,10.1186/s12911-019-0777-0
3,33059366,The UMLS knowledge sources at 30: indispensabl...,"Humphreys BL, Del Fiol G, Xu H.",J Am Med Inform Assoc. 2020 Oct 1;27(10):1499-...,Humphreys BL,J Am Med Inform Assoc,2020,2020/10/15,PMC7647371,NaN,10.1093/jamia/ocaa208
4,25810773,The CHEMDNER corpus of chemicals and drugs and...,"Krallinger M, Rabal O, Leitner F, Vazquez M, S...",J Cheminform. 2015 Jan 19;7(Suppl 1 Text minin...,Krallinger M,J Cheminform,2015,2015/03/27,PMC4331692,NaN,10.1186/1758-2946-7-S1-S2


In [29]:
def make_citation(r):
    base = "{authors} {title}. {citation}. PMID: {pmid}".format(
        authors=r['Authors'] if r['Authors'].endswith('.') else r['Authors']+'.',
        title=r['Title'],
        citation=r['Citation'],
        pmid=r['PMID']
    )

    if not pd.isna(r['PMCID']): base += '; PMCID: %s' % r['PMCID']
    if not pd.isna(r['DOI']): base += '; DOI: %s' % r['DOI']

    return base

In [35]:
df['Publication Year'].max()

2023

In [36]:
from datetime import datetime
from io import StringIO
buf = StringIO()

def print_buf(buf, s):
    buf.write(s)
    buf.write('\n')
    print(s)

print_buf(buf, """Title: Publications
Date: %s
Slug: publications
Summary: Publications
""" % (datetime.now().strftime('%Y-%m-%d'))
)
year_max = df['Publication Year'].max()
year_min = df['Publication Year'].min()

for year in range(year_max, year_min-1, -1):
    dft = df[df['Publication Year']==year].reset_index(drop=True)
    print_buf(buf, '## %s' % year)
    for idx, row in dft.iterrows():
        print_buf(buf, "%s. %s" % (idx+1, make_citation(row)))
    print_buf(buf, '\n')

Title: Publications
Date: 2023-09-07
Slug: publications
Summary: Publications

## 2023
1. Dang Y, Li F, Hu X, Keloth VK, Zhang M, Fu S, Amith MF, Fan JW, Du J, Yu E, Liu H, Jiang X, Xu H, Tao C. Systematic design and data-driven evaluation of social determinants of health ontology (SDoHO). J Am Med Inform Assoc. 2023 Aug 18;30(9):1465-1473. doi: 10.1093/jamia/ocad096.. PMID: 37301740; PMCID: PMC10436148; DOI: 10.1093/jamia/ocad096
2. Hawrylycz M, Martone ME, Ascoli GA, Bjaalie JG, Dong HW, Ghosh SS, Gillis J, Hertzano R, Haynor DR, Hof PR, Kim Y, Lein E, Liu Y, Miller JA, Mitra PP, Mukamel E, Ng L, Osumi-Sutherland D, Peng H, Ray PL, Sanchez R, Regev A, Ropelewski A, Scheuermann RH, Tan SZK, Thompson CL, Tickle T, Tilgner H, Varghese M, Wester B, White O, Zeng H, Aevermann B, Allemang D, Ament S, Athey TL, Baker C, Baker KS, Baker PM, Bandrowski A, Banerjee S, Bishwakarma P, Carr A, Chen M, Choudhury R, Cool J, Creasy H, D'Orazi F, Degatano K, Dichter B, Ding SL, Dolbeare T, Ecker JR, 

In [28]:
# save it
with open('publications.txt', 'w', encoding='utf8') as f:
    f.write(buffer.getvalue())

print('* done!')

* done!


# Parse grants

In [51]:
df = pd.read_csv('SearchResult_Export_07Sep2023_015921.csv', skiprows=5)
df.head(2).T

,0,1
NIH Spending Categorization,No NIH Category available,No NIH Category available
Project Terms,Address;Affect;Alternative Splicing;Astrocytes...,Agreement;Antibodies;Area;Award;Benchmarking;B...
Project Title,Immune checkpoints in the CNS and HIV-associat...,RADx-Rad Discoveries & Data: Consortium Coordi...
Public Health Relevance,PROJECT NARRATIVE\nOur research aims to study ...,NARRATIVE \n \nThe RADx-­rad Consortium Da...
Administering IC,NINDS,NLM
Application ID,10889463,10854683
Award Notice Date,8/29/2023 12:00:00 AM,6/2/2023 12:00:00 AM
Opportunity Number,PA-20-185,PA-21-268
Project Number,1R56NS129438-01A1,3U24LM013755-03S2
Type,1,3


In [64]:
tpl = """### {title}
- Project Number: [{pn}]({url})
- Start Date: {start_date}
- End Date: {end_date}

{abstract}

---
"""

tpl = """
- [{pn}]({url}) ({start_date} - {end_date}): {title}
"""
buf = StringIO()
pns = []
def print_buf(buf, s):
    buf.write(s)
    buf.write('\n')
    print(s)
    
for idx, row in df.iterrows():
    pn = row['Project Number'].split('-')[0][1:]
    if pn in pns: continue
    pns.append(pn)

    abstract = row['Project Abstract'].replace('\n', ' ')
    
    print_buf(buf, tpl.format(
        title = row['Project Title'],
        url = "https://reporter.nih.gov/project-details/%s" % (row['Application ID']),
        pn = row['Project Number'].split('-')[0][1:],
        start_date = row['Project Start Date'].split(' ')[0],
        end_date = row['Project End Date'].split(' ')[0],
        abstract = abstract,
    ))

with open('grants.txt', 'w', encoding='utf8') as f:
    f.write(buf.getvalue())

print('* done!')


- [R56NS129438](https://reporter.nih.gov/project-details/10889463) (9/1/2023 - 8/31/2024): Immune checkpoints in the CNS and HIV-associated neurocognitive disorder


- [U24LM013755](https://reporter.nih.gov/project-details/10854683) (12/21/2020 - 11/30/2024): RADx-Rad Discoveries & Data: Consortium Coordination Center Program Organization


- [R01LM013519](https://reporter.nih.gov/project-details/10839649) (9/1/2021 - 5/31/2025): PheBC: bias correction methods for EHR derived phenotype


- [R01MH133474](https://reporter.nih.gov/project-details/10700531) (2/16/2023 - 1/31/2028): Early prevention interventions towards ART-free pediatric HIV remission


- [U24MH130988](https://reporter.nih.gov/project-details/10695226) (9/1/2022 - 6/30/2027): Engagement and outreach to achieve a FAIR data ecosystem for the BICAN


- [R01AG078154](https://reporter.nih.gov/project-details/10689949) (9/1/2022 - 5/31/2027): Detecting synergistic effects of pharmacological and non-pharmacological intervention